In [3]:
import numpy as np
import spacy
import pandas as pd

file = pd.read_csv('newegg.csv')
df_description = pd.DataFrame(data=file, columns=['brand_name','items_Description','ratings','prices','Category'])
# print(df_description.items_Description)
nlp = spacy.load("en_core_web_md")
tuple_sentences = tuple(df_description['items_Description'])
sentence_vectors = [nlp(sentence).vector for sentence in tuple_sentences]
df_description.drop(columns=['brand_name','ratings','prices','Category'],axis=1,inplace=True)
print(df_description)

                                      items_Description
0     CORSAIR Vengeance RGB Pro 16GB (2 x 8GB) 288-P...
1     Kingston 16GB (2 x 8GB) 240-Pin DDR3 SDRAM DDR...
2     CORSAIR Vengeance LPX 32GB (4 x 8GB) 288-Pin D...
3     CORSAIR Vengeance LPX 128GB (4 x 32GB) 288-Pin...
4     AMD Ryzen Threadripper 3990X 64-Core 2.9 GHz S...
...                                                 ...
2700  EVGA SuperNOVA 1000 T2 220-T2-1000-X1 80+ TITA...
2701  Thermaltake TR2 TR-600 600W ATX12V v2.3 SLI Re...
2702  EVGA SuperNOVA 650 P2 220-P2-0650-X1 80+ PLATI...
2703  EVGA SuperNOVA 850 PQ, 80 Plus PLATINUM 850W, ...
2704  Thermaltake Toughpower PS-TPD-0850MPCGUS-1 850...

[2705 rows x 1 columns]


In [4]:
print(len(sentence_vectors))

2705


In [5]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

# Number of clusters
num_clusters = 7
sentence_vectors = np.array(sentence_vectors).reshape(2705,300,)
# Perform K-means clustering
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(sentence_vectors)

#Test the string you want to
#First vectorize using spacy
#use kmeans.predict to test that vectorized description

string = "" #from the product page
num_of_neighbours = 5
test_data = []
string_vectorized = nlp(string).vector
test_data.append(string_vectorized)
test_data = np.array(test_data)
test_cluster = kmeans.predict(test_data.reshape(1,-1))


# Get the indices of data points in the same cluster
cluster_indices = np.where(kmeans.labels_ == test_cluster)[0]

# Calculate distances between the test data point and all data points in the cluster
distances = np.linalg.norm(sentence_vectors[cluster_indices] - test_data, axis=1)

# Find the closest data points (nearest neighbors)
closest_indices = cluster_indices[np.argsort(distances)[:num_of_neighbours]]

# Print the indices of the closest data points
print("Indices of Closest Data Points:", closest_indices)

for i in closest_indices:
    print(df_description.iloc[i]['items_Description'] + '\n')

Indices of Closest Data Points: [1809 1483 1846   53 2019]
Lenovo 60G1MAR2US ThinkVision T2054p

Lenovo ThinkVision T1714p 60FELAR1US UPS Battery

Lenovo ThinkVision Monitor 61ABMAR1US Monitor

Intel Xeon Gold 6234 8Core 3.30GHz Processor LGA-3647 OEM/4 Pack CD8069504283304

DELL ENTERPRISE ACCESSORIES 400-ATJG 6GBPS 2.5IN 7200RPM SERIAL ATA

